# Bibliotecas e funções auxiliares

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os.path as osp
import pandas as pd
import re

from itertools import pairwise

In [ ]:
DEFAULT_BINS = [0,1,2,5,10]

In [ ]:
def clean_models(models: list):
    stop_words = ['+',  '120hz', '2', '4g', '5g', 'amarelo', 'azul',
                  'basic', 'bluetooth', 'br', 'branco', 'cinza', 'de',
                  'earbuds', 'earphone', 'earphones', 'escuro', 'fone',
                  'gradiente', 'inteligente', 'laranja', 'mi', 'prata',
                  'preto', 'pulseira', 'rosa', 'roxo', 'tela', 'true',
                  'verde', 'wireless', 'wireless']

    pattern = '(([\d]+gb)?[\d]+gb)|(\([\d\w-]+\))|(\d+,\d+)|([+"\/\',’])'
    models = [model.lower() for model in models]
    models = [re.sub(pattern,'',model) for model in models]
    models = list(set([' '.join([token for token in model.split() if token not in stop_words]) for model in models]))
    
    return models

In [ ]:
def plot_results_count(search_results_count,bins=DEFAULT_BINS):
    heights,_ = np.histogram(search_results_count,bins=bins)
    x_labels = ['[{}-{})'.format(start,stop) for start,stop in pairwise(bins)]
    x_labels[-1] = x_labels[-1].replace(')',']')
    
    bar_colors = ['tab:blue' for x in range(len(heights))]
    bar_colors[0] = 'tab:red'
    bar_colors[heights.argmax()] = 'tab:orange'
    
    fig, ax = plt.subplots()
    
    for i in range(len(heights)):
        p = ax.bar(x_labels[i],heights[i], color=bar_colors[i], alpha=0.85)
        ax.bar_label(p, label_type='center')
    
    # remove all the ticks (both axes), and tick labels on the Y axis
    ax.tick_params(bottom=False, left=False,
                   labelbottom=True, labelleft=False)
    
    # remove the frame of the chart
    ax.spines['left'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
    
    ax.set_ylabel('Quantidade por Faixa')
    ax.set_xlabel('Faixa de Resultados')
    ax.set_title('Quantidade de Resultados Obtidos')
    
    plt.show()


# Carga e preparação dos dados

## Importação dos arquivos

In [ ]:
# importar tabela completa do SCH (Dados Abertos)
file_sch = 'datasets/produtos_certificados.zip'
usecols = [0, 1, 2, 3, 11, 12, 13, 14, 15]
dtype = {'Número de Homologação': 'str',
         'CNPJ do Solicitante': 'str'}
df_sch = pd.read_csv(file_sch,sep=';',usecols=usecols,dtype=dtype)

# contar quantidade de números de homologação
df_sch['Quantidade de Número de Homologação']=df_sch[['Número de Homologação','Modelo']].groupby('Número de Homologação').transform('count')
df_sch = df_sch.sort_values(by=['Quantidade de Número de Homologação','Data da Homologação'],ascending=False).reset_index(drop=True)
df_sch.head()

In [ ]:
# importar subset do SCH
file_subset_sch = 'datasets/subset_sch.txt'
dtype = {'Número de Homologação': 'str'}
df_subset_sch = pd.read_csv(file_subset_sch,names=['Número de Homologação'],dtype=dtype)

# merge SCH subset and SCH to join columns "Modelo" and "Nome Comercial"
columns_to_merge = ['Número de Homologação', 'Modelo', 'Nome Comercial']
df_subset_sch=df_subset_sch.merge(df_sch[columns_to_merge])
df_subset_sch = df_subset_sch.fillna('#NULO#')
df_subset_sch = df_subset_sch.drop_duplicates()
df_subset_sch.head()

In [ ]:
# importa lista de EAN da Xiaomi
file_ean_xiaomi = 'datasets/ean_xiaomi.xlsx'
dtype = {'EAN': 'str'}
df_ean_xiaomi = pd.read_excel(file_ean_xiaomi,dtype=dtype)
df_ean_xiaomi.head()

In [ ]:
ean_to_search = df_ean_xiaomi['EAN']
ean_to_search.name='original_query'

ean_models_to_search = pd.Series(clean_models(df_ean_xiaomi['Descrição']))
ean_models_to_search.name='original_query'

sch_to_search = df_subset_sch['Número de Homologação'].drop_duplicates()
sch_to_search.name='original_query'

sch_models_to_search = pd.Series(list(set(df_subset_sch['Modelo'].unique().tolist()+df_subset_sch['Nome Comercial'].unique().tolist())))
sch_models_to_search.name='original_query'

In [ ]:
# carregar resultados de pesquisa
file_search_results = 'datasets/search_results/products_search_results.parquet'
df_search_results = pd.read_parquet(file_search_results)

# dividir resultados de pesquisa em grupos de pesquisa
file_search_results_ean = 'datasets/search_results/products_search_results_ean.parquet'
file_search_results_ean_models = 'datasets/search_results/products_search_results_ean_models.parquet'
file_search_results_sch = 'datasets/search_results/products_search_results_sch.parquet'
file_search_results_sch_models = 'datasets/search_results/products_search_results_sch_models.parquet'

# carregar os arquivos por grupo, se já tiverem sido separados anteriormente,
# caso contrário, separar e salvar o arquivo correspondente
if osp.exists(file_search_results_ean):
    df_search_results_ean = pd.read_parquet(file_search_results_ean)    
    print(f'File {file_search_results_ean} loaded')
else:
    df_search_results_ean = df_search_results.merge(ean_to_search)
    df_search_results_ean.to_parquet(file_search_results_ean)
    print(f'Dataframe df_search_results_sch created and saved to file {file_search_results_ean}')

if osp.exists(file_search_results_ean_models):
    df_search_results_ean_models = pd.read_parquet(file_search_results_ean_models)  
    print(f'File {file_search_results_ean_models} loaded')
else:
    df_search_results_ean_models = df_search_results.merge(ean_models_to_search)
    df_search_results_ean_models.to_parquet(file_search_results_ean_models)
    print(f'Dataframe df_search_results_sch created and saved to file {file_search_results_ean_models}')

if osp.exists(file_search_results_sch):
    df_search_results_sch = pd.read_parquet(file_search_results_sch)   
    print(f'File {file_search_results_sch} loaded')
else:
    df_search_results_sch = df_search_results.merge(sch_to_search)
    df_search_results_sch.to_parquet(file_search_results_sch)
    print(f'Dataframe df_search_results_sch created and saved to file {file_search_results_sch}')

if osp.exists(file_search_results_sch_models):
    df_search_results_sch_models = pd.read_parquet(file_search_results_sch_models)   
    print(f'File {file_search_results_sch_models} loaded')
else:
    df_search_results_sch_models = df_search_results.merge(sch_models_to_search)
    df_search_results_sch_models.to_parquet(file_search_results_sch_models)
    print(f'Dataframe df_search_results_sch created and saved to file {file_search_results_sch_models}')

# Análise

In [ ]:
df_search_results_ean['results'] = df_search_results_ean['url'].apply(lambda x: 0 if x is None else 1)
df_search_results_ean

In [ ]:
search_results_count = df_search_results_ean[['original_query','results']].groupby('original_query').sum().values.reshape(-1,)
plot_results_count(search_results_count)